### Setup and Configuration

In [ ]:
# pip installs

%pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
%pip install -q --upgrade requests==2.32.4 bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0 peft==0.14.0 trl==0.14.0 fsspec==2025.3.0 matplotlib wandb datasets

print("Installation successfull.")

In [ ]:
# imports

import os
import re
import math
from tqdm import tqdm
from google.colab import userdata
from huggingface_hub import login
import torch
import torch.nn.functional as F
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, set_seed
from datasets import load_dataset, Dataset, DatasetDict
from datetime import datetime
from peft import PeftModel
import matplotlib.pyplot as plt

In [ ]:
#constants
QUANT_4BIT = True
BASE_MODEL = "meta-llama/Llama-3.1-8B-Instruct"
DATASET_NAME = "Vishy08/pricer-data"

In [ ]:
hf_token = userdata.get("HF_TOKEN")

login(hf_token,add_to_git_credential = True )

### Loading the model and data

In [ ]:
#loading the data
dataset = load_dataset(DATASET_NAME)
train = dataset['train']
test = dataset['test']
val = dataset['val']

In [ ]:
if QUANT_4BIT:
  quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
  )
else:
  quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.bfloat16
  )

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True,device_map="auto")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
#loading quantized base model
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL ,
    quantization_config = quant_config ,
    device_map ="auto"
)

base_model.generation_config.pad_token_id = tokenizer.pad_token_id

fine_tuned_model = PeftModel.from_pretrained(base_model, "Vishy08/product-pricer-08-12-2025_04.35.08")

print(f"Memory footprint: {fine_tuned_model.get_memory_footprint() / 1e6:.1f} MB")

In [ ]:
fine_tuned_model


### Inference and Testing

In [ ]:
def extract_pricer(s):
    if "Price is $" in s:
        # Isolating the part AFTER the label
        text_after = s.split("Price is $")[1]
        clean_text = text_after.replace(',', '')
        match = re.search(r"[-+]?(?:\d+\.?\d*|\.\d+)", clean_text)
        if match:
            try:
                return float(match.group())
            except ValueError:
                return 0.0

    return 0.0

In [ ]:
extract_pricer("Price is $799.9. fuck you bitch")

In [ ]:
def model_predict(datapoint):
    set_seed(42)
    prompt = datapoint['test_prompt']
    inputs = tokenizer.encode(prompt , return_tensors="pt").to("cuda")
    attention_mask = torch.ones(inputs.shape , device = 'cuda')
    outputs = fine_tuned_model.generate(inputs ,
                                        attention_mask = attention_mask ,
                                        max_new_tokens = 3 ,
                                        num_return_sequences = 1)


    response = tokenizer.decode(outputs[0])
    return extract_pricer(response)

#### Tester Class

In [ ]:
import math
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import numpy as np

# COLOR MAP for terminal output
GREEN = "\033[92m"
ORANGE = "\033[93m"
RED = "\033[91m"
RESET = "\033[0m"
COLOR_MAP = {"red": RED, "orange": ORANGE, "green": GREEN}

class Tester:
    def __init__(self, predictor, data, title=None, size=250):
        self.predictor = predictor
        self.data = data
        self.title = title or predictor.__name__.replace("_", " ").title()
        self.size = size

        # Use numpy arrays for numerical data
        self.guesses = np.zeros(size)
        self.truths = np.zeros(size)
        self.errors = np.zeros(size)
        self.lche = np.zeros(size)
        self.sles = np.zeros(size)
        self.colors = []  # Keep as list for strings

        # Counters
        self.green_count = 0
        self.orange_count = 0
        self.red_count = 0

        # Cache for computed metrics
        self._average_error = None
        self._rmsle = None

    def run_datapoint(self, i):
        try:
            datapoint = self.data[i] # Changed .iloc[i] to [i] for datasets.Dataset objects
            guess = float(self.predictor(datapoint)) # Pass the entire datapoint object
            truth = float(datapoint['price'])

            error = abs(truth - guess)
            log_error = math.log(truth + 1) - math.log(guess + 1)
            sle = log_error ** 2
            log_cosh_error = self.safe_log_cosh(error)

            color = self.color_for(error, truth)

            # Update counters
            if color == 'green':
                self.green_count += 1
            elif color == 'orange':
                self.orange_count += 1
            else:  # red
                self.red_count += 1

            # Store results in arrays
            self.guesses[i] = guess
            self.truths[i] = truth
            self.errors[i] = error
            self.lche[i] = log_cosh_error
            self.sles[i] = sle
            self.colors.append(color)

            return color

        except (ValueError, TypeError, AttributeError) as e:
            print(f"Error processing datapoint {i}: {e}")
            self.colors.append('red')
            self.red_count += 1
            return 'red'

    def safe_log_cosh(self, x):
        """Avoids overflow in log cosh calculation"""
        x = max(min(x, 500), -500)  # Cap between -500 and 500
        return math.log(math.cosh(x))

    def color_for(self, error, truth):
        if error < 40 or error/truth < 0.2:
            return 'green'
        elif error < 80 or error/truth < 0.4:
            return 'orange'
        else:
            return 'red'

    @property
    def average_error(self):
        if self._average_error is None:
            self._average_error = np.mean(self.errors)
        return self._average_error

    @property
    def rmsle(self):
        if self._rmsle is None:
            self._rmsle = math.sqrt(np.mean(self.sles))
        return self._rmsle

    def chart(self, title):
        plt.figure(figsize=(12, 8))

        max_val = max(np.max(self.truths), np.max(self.guesses))

        # Add error bands
        x = np.linspace(0, max_val, 100)
        plt.fill_between(x, x*0.8, x*1.2, color='green', alpha=0.1, label='±20% Range')
        plt.fill_between(x, x*0.6, x*1.4, color='orange', alpha=0.1, label='±40% Range')

        # Perfect prediction line
        plt.plot([0, max_val], [0, max_val], color='skyblue', lw=2,
                alpha=0.6, label='Perfect Prediction')

        # Scatter plot
        # Use self.colors directly as it already contains valid matplotlib color strings
        plt.scatter(self.truths, self.guesses, s=50, c=self.colors,
                   alpha=0.6, label='Predictions')

        # Statistics text box
        green_pct = (self.green_count/self.size*100)
        orange_pct = (self.orange_count/self.size*100)
        red_pct = (self.red_count/self.size*100)

        stats_text = (
            f'Accuracy Distribution:\n'
            f'Green: {green_pct:.1f}%\n'
            f'Orange: {orange_pct:.1f}%\n'
            f'Red: {red_pct:.1f}%\n'
            f'Average Error: ${self.average_error:,.2f}\n'
            f'RMSLE: {self.rmsle:.2f}'
        )

        plt.text(0.02, 0.98, stats_text,
                transform=plt.gca().transAxes,
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

        # Customize plot
        plt.xlabel('True Values ($)', fontsize=12)
        plt.ylabel('Predicted Values ($)', fontsize=12)
        plt.title(title, fontsize=14, pad=20)
        plt.xlim(0, max_val)
        plt.ylim(0, max_val)
        plt.grid(True, alpha=0.3)
        plt.legend(loc='lower right')

        plt.tight_layout()
        plt.show()

    def report(self):
        # Print summary statistics with color
        print("\nTest Results Summary:")
        print(f"Total Predictions: {self.size}")
        print(f"{GREEN}Correct (Green): {self.green_count} ({self.green_count/self.size*100:.1f}%){RESET}")
        print(f"{ORANGE}Close (Orange): {self.orange_count} ({self.orange_count/self.size*100:.1f}%){RESET}")
        print(f"{RED}Wrong (Red): {self.red_count} ({self.red_count/self.size*100:.1f}%){RESET}")
        print(f"Average Error: ${self.average_error:,.2f}")
        print(f"RMSLE: {self.rmsle:.2f}")

        title = f"{self.title} Error=${self.average_error:,.2f}  RMSLE={self.rmsle:.2f}  HIT={self.green_count/self.size*100:.1f}%"
        self.chart(title)

    def run(self):
        # Progress bar for overall testing
        with tqdm(total=self.size, desc="Testing Progress",
                 bar_format="{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]") as pbar:

            # Progress bar for correct predictions
            expected_correct = int(self.size * 0.7)  # Set expected correct to 70%
            with tqdm(total=expected_correct, desc="Correct Predictions",
                     bar_format="{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt}") as correct_pbar:

                for i in range(self.size):
                    color = self.run_datapoint(i)
                    pbar.update(1)

                    if color == 'green' and correct_pbar.n < expected_correct:
                        correct_pbar.update(1)

        self.report()

    @classmethod
    def test(cls, function, data):
        cls(function, data).run()

#### Testing

In [ ]:
Tester.test(model_predict , test)